# Exemple de notebook - filtres sql et récupération des données en pandas

L'objectif de ce notebook est de fournir des exemples pour pré-filtrer les données via sql avant de charger les données dans un DataFrame pandas.


In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [6]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données.

import duckdb

ODIS_DUCKDB_FILE = "odis.duckdb"
PCC_DUCKDB_FILE = "dev.duckdb"

con = duckdb.connect(database=PCC_DUCKDB_FILE, read_only=True)
con.sql(f"ATTACH '{ODIS_DUCKDB_FILE}' AS odis;")

## Filtres

1. Filtrer les cat nat publiées depuis 2000


In [7]:
query_2020 = """
SELECT
	*
FROM dev.main.catnat_gaspar
WHERE dat_pub_arrete >= '2000-01-01'
"""

cat_nat_2000 = con.sql(query_2020)
cat_nat_2000_df = cat_nat_2000.df()
cat_nat_2000_df.head(2)

,cod_nat_catnat,cod_commune,lib_commune,num_risque_jo,lib_risque_jo,dat_deb,dat_fin,dat_pub_arrete,dat_pub_jo,dat_maj
0,INTE0000044A,97130,Terre-de-Bas,CMV,Chocs Mécaniques liés à l'action des Vagues,1999-11-18,1999-11-19,2000-02-14,2000-03-03,2022-05-24 11:23:51.298
1,INTE0000044A,97131,Terre-de-Haut,CMV,Chocs Mécaniques liés à l'action des Vagues,1999-11-18,1999-11-19,2000-02-14,2000-03-03,2022-05-24 11:23:51.298


2. Filtrer mes donées pour en récupérer une partie


In [16]:
where_clause = """
"YEAR" == '2022'
"""
query_2022 = f"""
SELECT
  *
FROM odis.main."gold_gold_logements_territoires"
WHERE
  {where_clause}
"""

logements_2022 = con.sql(query_2022)
logements_2022_df = logements_2022.df()
logements_2022_df.head(2)

,codgeo,YEAR,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RPMAISON,RPAPPART,NB_MOY_PIECE,MEN,NBPI_RP
0,01001,2022.0,379.0,77.0,11.0,14.0,369.0,9.0,345.0,9.0,23.19481,77.0,1786.0
1,01002,2022.0,175.0,63.0,41.0,13.0,173.0,2.0,119.0,2.0,9.77778,63.0,616.0


## Selectionner des colonnes avant d'exécuter la requête


Selectionner les colonnes avant de charger les données permets une exécution plus rapide et limite l'usage de la mémoire.


In [17]:
where_clause = """
"YEAR" == '2022'
"""
query_rp = f"""
SELECT
  codgeo,
  RP as nombre_de_residences_principales
FROM odis.main."gold_gold_logements_territoires"
WHERE
  {where_clause}
"""
residences_principales_2022 = con.sql(query_rp)
residences_principales_2022_df = residences_principales_2022.df()
residences_principales_2022_df.head(2)

,codgeo,nombre_de_residences_principales
0,01001,77.0
1,01002,63.0


## Jointure

Joindre edc_prelevements et edc_resultats sur referenceprel pour obtenir les résultats associés à chaque prélèvement :


In [ ]:
query = f"""
SELECT
  "edc_prelevements"."referenceprel",
  "edc_prelevements"."dateprel",
  "edc_prelevements"."nomcommuneprinc",
  "edc_resultats"."libmajparametre",
  "edc_resultats"."insituana",
  "edc_resultats"."rqana",
  "edc_resultats"."cdunitereferencesiseeaux"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) AS edc_prelevements
INNER JOIN "edc_resultats"
  ON "edc_prelevements"."referenceprel" = "edc_resultats"."referenceprel"
"""


joined = con.sql(query)
joined_df = joined.df()
joined_df

## Groupby et aggregats

Nombre total de prélèvements non conforme par commune en 2024


In [ ]:
query = f"""
SELECT
  "nomcommuneprinc",
  COUNT("referenceprel") AS "nb_prelevements_non_conformes"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) 
GROUP BY
  1
"""
grouped = con.sql(query)
grouped_df = grouped.df()
grouped_df.sort_values("nb_prelevements_non_conformes", ascending=False)

## Autres exemples :


In [ ]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb
# Faisons une requête SQL en utilisant duckdb via la librarie python pour lister les substances qui ont été recherchées
# et les trier par ordre décroissant de leur nombre d'occurrences

con.sql("""
    SELECT libmajparametre, COUNT(*) as count
    FROM edc_resultats
    GROUP BY libmajparametre
    ORDER BY count DESC
""").show()

In [ ]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb

# Enfin, terminons par lister les prélèvements effectués dans une commune donnée

nomcommune = "TOULOUSE"

con.sql(f"""
    SELECT *
    FROM edc_prelevements
    WHERE nomcommuneprinc = '{nomcommune}'
""").show()